# IBM Coursera_Capstone

##### By: Amar Jusufagic

Write some markdown to explain that this notebook will be mainly used for the capstone project.
Import the pandas library as pd.
Import the Numpy library as np.
Print the following the statement: Hello Capstone Project Course!

In [1]:
# import important libraries

import pandas as pd
import numpy as np

print('Hello Capstone Project Course!')

Hello Capstone Project Course!


### PART 1

# Define DataFrame

### Use BeautifulSoup & Pandas to read_html, html = the website

In [2]:
import requests
from bs4 import BeautifulSoup

#res = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
#soup = BeautifulSoup(res.content,'lxml')
#table = soup.find_all('table')[0] 
#df = pd.read_html(str(table), header=0)

#### Convert df to df[0] to format for Jupyter Notebooks

In [3]:
#df = df[0] # dont run more than once

In [4]:
#df.head(8)

### Alternatively... just use Pandas
to read the tables from a website

In [5]:
tables = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M', header=0)

tables = tables[0] # must [0] to get the format below

tables.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


##### Remove all Boroughs = 'Not assigned' with !=
index will be reset in the next step, so don't worry

In [6]:
tables = tables[tables.Borough != 'Not assigned']

tables.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor


##### Group Neighborhoods together with ',' that share the same Postcode and Borough

In [7]:
tables = tables.groupby('Postcode').agg({'Borough':'first', 
                             'Neighbourhood': ', '.join}).reset_index()

tables.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


##### Count the number of (rows, collumns)

In [8]:
tables.shape

(103, 3)

##### Check 'Neighbourhood' for any 'Not assigned' values

output will show any existing values (answer shows none)

In [9]:
unassigned = tables.loc[tables['Neighbourhood'] == 'Not assigned']

unassigned.head()

,Postcode,Borough,Neighbourhood


## Project Checkpoint Reached!!!

### PART 2

##### Install geocoder

In [27]:
# %pip install geocoder
# or

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.7.12
  latest version: 4.8.2

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /Users/amarica/opt/anaconda3

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    conda-4.8.2                |           py37_0         3.0 MB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-1.21.0               |             py_0          58 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         3.1 MB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.50-py_0
  geopy              conda-forge/noarch::geopy-1.21.0-py_

In [29]:
CLIENT_ID = 'AAZM52WB5PH5QDCBF21S4NWZ5BAEABLYJ03K2BVUDSA2MVEC' # your Foursquare ID
CLIENT_SECRET = 'JL5EF2NIMK3U1E2GSAWJVN5JBQVJXP4ZCEWJERC5OUSBZPUH' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: AAZM52WB5PH5QDCBF21S4NWZ5BAEABLYJ03K2BVUDSA2MVEC
CLIENT_SECRET:JL5EF2NIMK3U1E2GSAWJVN5JBQVJXP4ZCEWJERC5OUSBZPUH


#### import geocoder 
prepare long and lat loading

### THIS SECTION WAS MARKDOWN BECAUSE IT WAS UNABLE TO RUN GEOCODER

postal_code = tables['Postcode']

import geocoder # import geocoder

#### initialize your variable to None
lat_lng_coords = None

#### loop until you get the coordinates
while(lat_lng_coords is None):
  g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
  lat_lng_coords = g.latlng

latitude = lat_lng_coords[0]
longitude = lat_lng_coords[1]

print(latitude, longitude)

address = ('{}, Toronto, Ontario'.format(postal_code))

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

### Since Geocoder refuses to work for us,
we'll go ahead and import the csv with the data and merge with our existing dataframe

##### Import Geospatial_Coordinates.csv with pandas

In [40]:
long_lat = pd.read_csv('Geospatial_Coordinates.csv')

long_lat.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


We must convert Postal Code to Postcode in order to merge our data frames

In [49]:
long_lat = long_lat.rename(columns={'Postal Code':'Postcode'})

long_lat.head()

,Postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


##### AWESOME!
Now time to merge() tables & long_lat using a common column 

In [58]:
geo = pd.merge(tables, long_lat, on='Postcode')

geo.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


# SiiiiiCKKKK